In [1]:
!pip install python-dotenv huggingface_hub bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
# Configuration and imports
import os
import yaml
import json
import pandas as pd
from typing import Dict, List, Optional, Tuple
from dotenv import load_dotenv
from datetime import datetime
import logging
from huggingface_hub.hf_api import HfFolder
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextGenerationPipeline
from utils import email_data_validation

In [10]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
HfFolder.save_token(hf_token)

# Set the quantization configuration for LLaMA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16"
)

# Load the config and the prompts
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

with open("prompts.json", "r") as f:
    prompts = json.load(f)

email_classification_prompt = prompts["email_classification"][config["email_prompt"]]
response_generation_prompt = prompts["reponse_generation"][config["generation_prompt"]]

In [11]:
# Sample email dataset
sample_emails = [
    {
        "id": "001",
        "from": "angry.customer@example.com",
        "subject": "Broken product received",
        "body": "I received my order #12345 yesterday but it arrived completely damaged. This is unacceptable and I demand a refund immediately. This is the worst customer service I've experienced.",
        "timestamp": "2024-03-15T10:30:00Z"
    },
    {
        "id": "002",
        "from": "curious.shopper@example.com",
        "subject": "Question about product specifications",
        "body": "Hi, I'm interested in buying your premium package but I couldn't find information about whether it's compatible with Mac OS. Could you please clarify this? Thanks!",
        "timestamp": "2024-03-15T11:45:00Z"
    },
    {
        "id": "003",
        "from": "happy.user@example.com",
        "subject": "Amazing customer support",
        "body": "I just wanted to say thank you for the excellent support I received from Sarah on your team. She went above and beyond to help resolve my issue. Keep up the great work!",
        "timestamp": "2024-03-15T13:15:00Z"
    },
    {
        "id": "004",
        "from": "tech.user@example.com",
        "subject": "Need help with installation",
        "body": "I've been trying to install the software for the past hour but keep getting error code 5123. I've already tried restarting my computer and clearing the cache. Please help!",
        "timestamp": "2024-03-15T14:20:00Z"
    },
    {
        "id": "005",
        "from": "business.client@example.com",
        "subject": "Partnership opportunity",
        "body": "Our company is interested in exploring potential partnership opportunities with your organization. Would it be possible to schedule a call next week to discuss this further?",
        "timestamp": "2024-03-15T15:00:00Z"
    }
]

In [ ]:
class EmailProcessor:
    def __init__(self):
        """Initialize the email processor with LLM API key."""

        model_name = config["llm_model"]
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
        self.pipeline = TextGenerationPipeline(model=self.model, tokenizer=self.tokenizer)

        # Define valid categories
        self.valid_categories = {
            "complaint", "inquiry", "feedback",
            "support_request", "other"
        }

    def classify_email(self, email: Dict) -> Optional[str]:
        """
        Classify an email using LLM.
        Returns the classification category or None if classification fails.
        """

        system_prompt = email_classification_prompt
        user_prompt = f"Subject: {email['subject']}, Body: {email['body']}"

        prompt = [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ]

        try:
            response = self.pipeline(prompt, max_new_tokens=20)[0]["generated_text"]
            predicted_category = response[-1]["content"]

            if predicted_category in self.valid_categories:
                return predicted_category

            return "other"

        except Exception as e:
            logger.error(f"Classification failed for email {email['id']}: {e}")
            return None

    def generate_response(self, email: Dict, classification: str) -> Optional[str]:
        """
        Generate an automated response based on email classification.
        """

        system_prompt = response_generation_prompt

        user_prompt = f"Subject: {email['subject']}, Body: {email['body']}, Category: {classification}"

        prompt = [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ]

        try:
            response = self.pipeline(prompt, max_new_tokens=200)[0]["generated_text"]
            generated_response = response[-1]["content"]
            print(generated_response)
            return generated_response

        except Exception as e:
            logger.error(f"Response generation failed for email {email['id']}: {e}")
            return None

In [ ]:
class EmailAutomationSystem:
    def __init__(self, processor: EmailProcessor):
        """Initialize the automation system with an EmailProcessor."""
        self.processor = processor
        self.response_handlers = {
            "complaint": self._handle_complaint,
            "inquiry": self._handle_inquiry,
            "feedback": self._handle_feedback,
            "support_request": self._handle_support_request,
            "other": self._handle_other
        }

    def process_email(self, email: Dict) -> Dict:
        """
        Process a single email through the complete pipeline.
        Returns a dictionary with the processing results.
        """

        try:
            classification = self.processor.classify_email(email)

            if not classification:
                raise ValueError(f"Failed to classify email {email['id']}")

            handler = self.response_handlers.get(classification, self._handle_other)

            response = self.processor.generate_response(email, classification)
            if not response:
                raise ValueError(f"Failed to generate response for email {email['id']}")

            if classification == "complaint":
                send_complaint_response(email["from"], response)
            else:
                send_standard_response(email["from"], response)

            handler(email)

            results = {
                "id": email["id"],
                "email_id": email["from"],
                "success": True,
                "classification": classification,
                "response_sent": response
            }

        except Exception as e:
            results = {
                "id": email["id"],
                "email_id": email["from"],
                "success": False,
                "classification": None,
                "response_sent": None
            }

        return results

    def _handle_complaint(self, email: Dict):
        """
        Handle complaint emails.
        """
        create_urgent_ticket(email["from"], "complaint", email["body"])

    def _handle_inquiry(self, email: Dict):
        """
        Handle inquiry emails.
        """
        create_support_ticket(email["from"], email["body"])

    def _handle_feedback(self, email: Dict):
        """
        Handle feedback emails.
        """
        log_customer_feedback(email["from"], email["body"])

    def _handle_support_request(self, email: Dict):
        """
        Handle support request emails.
        """
        create_support_ticket(email["from"], email["body"])

    def _handle_other(self, email: Dict):
        """
        Handle other category emails.
        """
        logger.info(f"Handled email {email['id']}")

In [14]:
# Mock service functions
def send_complaint_response(email_id: str, response: str):
    """Mock function to simulate sending a response to a complaint"""
    logger.info(f"Sending complaint response for email {email_id}")
    # In real implementation: integrate with email service


def send_standard_response(email_id: str, response: str):
    """Mock function to simulate sending a standard response"""
    logger.info(f"Sending standard response for email {email_id}")
    # In real implementation: integrate with email service


def create_urgent_ticket(email_id: str, category: str, context: str):
    """Mock function to simulate creating an urgent ticket"""
    logger.info(f"Creating urgent ticket for email {email_id}")
    # In real implementation: integrate with ticket system


def create_support_ticket(email_id: str, context: str):
    """Mock function to simulate creating a support ticket"""
    logger.info(f"Creating support ticket for email {email_id}")
    # In real implementation: integrate with ticket system


def log_customer_feedback(email_id: str, feedback: str):
    """Mock function to simulate logging customer feedback"""
    logger.info(f"Logging feedback for email {email_id}")
    # In real implementation: integrate with feedback system

In [15]:
def run_demonstration():
    """Run a demonstration of the complete system."""
    # Initialize the system
    processor = EmailProcessor()
    automation_system = EmailAutomationSystem(processor)

    # Process all sample emails
    results = []
    for email in sample_emails:
        logger.info(f"\nValidating input data for email {email['id']}...")
        if not email_data_validation(email):
            raise ValueError("Given data has some missing fields!")
        logger.info(f"\nProcessing email {email['id']}...")
        result = automation_system.process_email(email)
        results.append(result)

    # Create a summary DataFrame
    df = pd.DataFrame(results)
    print("\nProcessing Summary:")
    print(df[["id", "email_id", "success", "classification", "response_sent"]])

    return df

In [16]:
results_df = run_demonstration()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Subject: Re: Damaged Product Received - Complaint Response

Dear [Customer's Name],

Thank you for bringing this issue to our attention. We apologize for the unacceptable condition of your order, which is completely unacceptable and not up to our quality standards. We have created an urgent ticket to address this matter and ensure a prompt resolution.

Please expect a refund for the damaged product, as well as any additional compensation for the inconvenience caused. Our dedicated team will review the situation and provide a full refund within the next 48 hours.

Your satisfaction is our top priority, and we appreciate your feedback. We will use this as an opportunity to review our packaging and shipping procedures to prevent such incidents in the future.

Thank you for your patience and understanding.

Best regards,
[Your Name]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Subject: Re: Question about product specifications

Dear [Customer],

Thank you for your interest in our premium package. I'm happy to help clarify the compatibility of our product with Mac OS. Our premium package is compatible with Mac OS, and we provide detailed specifications on our website. However, I'd be more than happy to provide you with the specific details you're looking for.

Please find the specifications here: [link to product specifications]. If you have any further questions or concerns, feel free to ask.

Best regards,
[Your Name]

A support ticket has been created for your inquiry.


...

Subject: Re: Amazing customer support

Dear [Customer],

Thank you so much for your kind words about your recent interaction with Sarah on our team! We're thrilled to hear that she was able to provide you with excellent support and resolve your issue to your satisfaction. We're proud of our team for going above and beyond to ensure our customers receive the best possible service.

We appreciate your feedback and will continue to strive for excellence in all our interactions.

Best regards,
[Your Name]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Subject: Re: Support Request - Error Code 5123

Dear [Customer Name],

Thank you for reaching out to us for assistance. I'm happy to help you resolve the issue with the software installation. I've created a support ticket for you, and I'll be happy to guide you through the troubleshooting process.

Can you please try updating the software to the latest version and then try reinstalling it? If the issue persists, we can explore further solutions together.

I'll be in touch with you soon to discuss the next steps.

Best regards,
[Your Name]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Subject: Re: Partnership Opportunity

Dear [Customer Name],

Thank you for your email expressing interest in a potential partnership opportunity with our company. We're excited to explore possibilities and would be delighted to schedule a call to discuss this further.

How about we schedule a call for Wednesday, [Date], at 2 PM EST or Thursday, [Date], at 10 AM EST? Please let us know which time slot works best for you, or suggest an alternative time that suits you.

We're looking forward to speaking with you and exploring ways we can collaborate.

Best regards,
[Your Name]

Processing Summary:
    id                     email_id  success   classification  \
0  001   angry.customer@example.com     True        complaint   
1  002  curious.shopper@example.com     True          inquiry   
2  003       happy.user@example.com     True         feedback   
3  004        tech.user@example.com     True  support_request   
4  005  business.client@example.com     True          inquiry   

       